# Chapter 6: Structure learning

In the previous lessons, we saw that causal inference aims to answer questions involving cause and effect. Specifically, we saw how Directed Acyclic Graphs (DAGs) could be used to incorporate our prior knowledge domain and effectively visualize the causal relationships in a clear-cut way.

DAGs represent causal structure by showing who causes who or who doesn’t cause who. A DAG is a special kind of graph where all edges are directed (information flow is in one direction) and no cycles exist (information that leaves a vertex cannot return to it). The vertices (circles) in a causal DAG represent variables, and edges (arrows) represent causation, where a variable is directly caused by its parents.

However, often in the real world, we may not be sure about which variables cause each other. 

**But how can we create a DAG?**
- Sometimes, we have prior knowledge, and we can use it to build a causal model
- Other times, we have limited knowledge and we want to estimate/retrieve the model from some data we have 

**With structure learning, we want to determine the structure of the graph that best captures the causal dependencies between the variables in the data set.** 
In other words, given a dataset, derive a causal model that describes it.


## How can we do it?

Unfortunately, there is no standard recipe for that, and that's why causal inference is generally challenging. 
Causal discovery is an example of an inverse problem. 
However, there are some techniques available, and here we are going to sketch the most common ones and key ideas.
The usual approach to solving inverse problems is to **make assumptions** about what you are trying to uncover. This narrows down the possible solutions and hopefully makes the problem solvable. 

There are four common assumptions made across causal discovery algorithms. A nice discussion of these assumptions can be found in Kalainathan et al (2018) https://arxiv.org/abs/1803.04929

- **Acyclicity** — causal structure can be represented by a DAG $G$
- **Markov Property** — all nodes are independent of their non-descendants when conditioned on their parents
- **Faithfulness** — all conditional independences in true underlying distribution $p$ are represented in $G$
- **Sufficiency** — any pair of nodes in $G$ has no common external cause

Although these assumptions help narrow down the number of possible models, they do not fully solve the problem. This is where a few tricks for causal discovery are helpful. A taxonomy of algorithms based on the following tricks is given in the figure below.

| **TRICK**                             | **ALGORITHM**                                               |
|-----------------------------------|---------------------------------------------------------|
| **Conditional Independence Testing**  | PC <br> Fast Causal Inference (FCI) <br>  Inductive Causation (IC) |
| **Greedy Search on DAG Space**        | Gready Equivalent Search (GES) <br>  Gready Interventional Equivalent Search (GIES) <br> Concave penalized Coordinate Descent with reparametrization (CCDr)                                                        |
| **Exploiting Asymmetry**              | Linear Non-Gaussian Acyclic Model (LINGAM) <br>  Nonlinear Additive Noise Models <br> Post_nonlinear Causal Model (PNL) <br> Granger Causality                                                      |
| **Hybrid**                            | Structural Agnostic Modeling (SAM) <br> Causal Additive Modeling (CAM) <br> Causal Generative Causal Neural Network (CGNN)                                                       |


### Trick 1: Conditional Independence Testing

One of these earliest causal discovery algorithms is the PC algorithm named after its authors Peter Spirtes and Clark Glymour. This algorithm (and others like it) use the idea that two statistically independent variables are not causally linked. The PC algorithm is illustrative of this first trick. An outline of the algorithm is given in the figure below (image taken from https://towardsdatascience.com/causal-discovery-6858f9af6dcb). 

![img](img/ch6/Trick1.png)

The first step is to form a fully connected, undirected graph using every variable in the dataset. Next, edges are deleted if the corresponding variables are independent. Then, connected edges undergo conditional independence testing e.g. independence test of bottom and far right node conditioned on middle node in the figure above (step 2).

If conditioning on a variable kills the dependence, that variable is added to the Separation set for those two variables. Depending on the size of the graph, conditional independence testing will continue (i.e. condition on more variables) until there are no more candidates for testing.

Next, colliders (i.e. X → Y ← Z) are oriented based on the Separation set of node pairs. Finally, remaining edges are directed based on 2 constraints, 1) no new v-structures and 2) no directed cycles can be formed.


### Trick 2: Greedy Search of Graph Space

A greedy algorithm is an approach for solving a problem by selecting the best option available at the moment. It doesn't worry whether the current best result will bring the overall optimal result.
The algorithm never reverses the earlier decision even if the choice is wrong. It works in a top-down approach.

Usually, greedy algorithms are easier to describe and can perform quite good compared to other algorithms.
However, greedy searches cannot guarantee an optimal solution. Neverthless, for most problems the space of possible DAGs is so big that finding a true optimal solution is intractable.

The Greedy Equivalence Search (GES) algorithm uses this trick. GES starts with an empty graph and iteratively adds directed edges such that the improvement in a model fitness measure (i.e. score) is maximized.

### Trick 3: Exploiting Asymmetries

As we say, fundamental property of causality is asymmetry. A could cause B, but B may not cause A. Thus, there are algorithms that leverage this idea to select between causal model candidates, usuallty with respect to time, complexity, and functional.

- Time asymmetry is quite natural. It’s the idea that causes happen before effects. This is expoited for example in the Granger causality test. We say that a variable X that evolves over time Granger-causes another evolving variable Y if predictions of the value of Y based on its own past values and on the past values of X are better than predictions of Y based only on Y's own past values.

- Complexity asymmetry follows the Occam’s razor principle, that simpler models are better. In other words, if you have handful of candidate models to choose from, this idea says to pick the simplest one. One way of quantifying simplicity (or complexity) is the Kolmogorov Complexity.

- Functional asymmetry assumes models that better fit a relationship are better candidates. For example, given two variables $X$ and $Y$, the nonlinear additive noise model (NANM) performs a nonlinear regression between $X$ and $Y$ , e.g. $y = f(x) + n$, where $n$ = noise/residual, in both directions. The model (i.e. causation) is then accepted if the potential cause (e.g. $x$) is independent of the noise term (e.g. $n$).

### Trick 4: Hybrid

The last part includes algorithms which differ one fron the other and exploit different assumtpions. For example, CGNN try to learn functional
causal models from observational data based on generative neural networks.  This area is pretty new and interested readers can find more techincal information in: 
[CGNN](https://arxiv.org/pdf/1711.08936.pdf)

## A practical exercise in Python

We will make use of the *bnlearn* library (built on top of the extensive *pgmpy* library) to perform structure learning and make predictions.

## References

[Review of Causal Discovery Methods Based on Graphical Models](https://www.frontiersin.org/articles/10.3389/fgene.2019.00524/full)